In [77]:
import pandas as pd
import numpy
import os
import sys
import math
from PIL import Image
from hilbertcurve.hilbertcurve import HilbertCurve

In [67]:
path = '/Users/aizazansari/Downloads/PLC_binaries/bin/'
path2 = '/Users/aizazansari/Downloads/PLC_binaries/annotations/'
arr = os.listdir(path)

In [72]:
def create_image(name,decimals):

    
    length = len(decimals)
    dimension = math.ceil(math.sqrt(length))
    bw_data = numpy.zeros((dimension, dimension), dtype=numpy.uint8)
    col_data = numpy.zeros((dimension, dimension, 3), dtype=numpy.uint8)
    
    for num in range(length):
        
        if decimals[num][-1]=='c':
            col = [0,255,0]
        else:
            col = [255,0,0]
        
        if (num//dimension%2==0):

            bw_data[num//dimension,num%dimension] = decimals[num][:-1]
            col_data[num//dimension,num%dimension] = col
        else:
            bw_data[num//dimension,dimension - num%dimension - 1]=decimals[num][:-1]
            col_data[num//dimension,dimension - num%dimension - 1]=col
            
    image = Image.fromarray(bw_data)
    image.save("images_plc/bw/"+name.strip('PRG')+"png",quality=100)
    image = Image.fromarray(col_data)
    image.save("images_plc/col/"+name.strip('PRG')+"png",quality=100)
 

In [74]:
for name in arr:
    decimals = []
    with open(path+name, mode='rb') as file:
        while 1:
            byte = file.read(1)
            if not byte:
                break
            else:
                decimals.append(int.from_bytes(byte, byteorder=sys.byteorder))
    with open(path2+name.strip('.PRG')+'.txt','r') as file:
        labels = file.readline()
        labels = labels.split('|')
        labels = labels[:len(labels)-1]
        
    data = [str(x[1])+x[0][0] for x in zip(labels,decimals)]
    create_image(name,data)


In [75]:
#resize images

path1 = '/Users/aizazansari/Desktop/Uni Data/MOMA/BinaryAnalysis/images_plc/bw/'
path2 = '/Users/aizazansari/Desktop/Uni Data/MOMA/BinaryAnalysis/images_plc/col/'

arr = os.listdir(path1)


for img in arr:
    if img.endswith('.png'):
        image1 = Image.open(path1+img)
        dim = pow(2,math.ceil(math.log(image1.size[0],2))) 
        ret_image1 = image1.resize([dim,dim], Image.NEAREST)
        ret_image1.save("images_plc/bw_up/"+ img,quality=100)

        image2 = Image.open(path2+img)
        ret_image2 = image2.resize([dim,dim], Image.NEAREST)
        ret_image2.save("images_plc/col_up/"+ img,quality=100)

In [78]:
def hc_coords(p, n,size):
    coords = []
    hilbert_curve = HilbertCurve(p, n)
    for i in range(size):
        coord = hilbert_curve.coordinates_from_distance(i)
        coords += [coord]
    return (coords)

In [79]:
def create_hilbert_image(name):
    
    bw_img = np.array(Image.open("images_plc/bw_up/"+name))
    col_img = np.array(Image.open("images_plc/col_up/"+name))
    
    dimension = bw_img.shape[0]
    coords = hc_dict[dimension]
 
    bw_data = numpy.zeros((dimension, dimension), dtype=numpy.uint8)
    col_data = numpy.zeros((dimension, dimension, 3), dtype=numpy.uint8)
    
    for num in range(dimension*dimension):
        if (num//dimension%2==0):
            bw_data[coords[num][0],coords[num][1]] = bw_img[num//dimension,num%dimension] 
            col_data[coords[num][0],coords[num][1]] = col_img[num//dimension,num%dimension] 
        else:
            bw_data[coords[num][0],coords[num][1]] = bw_img[num//dimension,dimension - num%dimension - 1]
            col_data[coords[num][0],coords[num][1]] = col_img[num//dimension,dimension - num%dimension - 1]
       
    image = Image.fromarray(bw_data)
    image.save("images_plc/bw_up_hilbert/"+name,quality=100)
    image = Image.fromarray(col_data)
    image.save("images_plc/col_up_hilbert/"+name,quality=100)

In [82]:
hc_dict = {8: hc_coords(3,2,8*8),
          16: hc_coords(4,2,16*16),
          32: hc_coords(5,2,32*32),
          64: hc_coords(6,2,64*64),
          128: hc_coords(7,2,128*128),
          256: hc_coords(8,2,256*256),
          512: hc_coords(9,2,512*512),
          1024: hc_coords(10,2,1024*1024)
          }

In [83]:
#generate hilbert images
arr = os.listdir('/Users/aizazansari/Desktop/Uni Data/MOMA/BinaryAnalysis/images_plc/bw_up')
for count, name in enumerate(arr):
    if name.endswith('.png'):
        create_hilbert_image(name)   


In [86]:
color2index = {(0, 0, 0): 0,
 (0, 255, 0): 1,
 (255, 0, 0): 2}

In [89]:
def rgb_to_index(name):
    rgb_data = np.array(Image.open('images_plc/col/'+name))
    dimension = rgb_data.shape[0]
    index_data = numpy.zeros((dimension, dimension), dtype=numpy.uint8)
    for i in range(dimension):
        for j in range(dimension):
            index_data[i][j] = color2index[tuple(rgb_data[i][j])]
    img = Image.fromarray(index_data)
    img.save('images_plc/col_index/'+name)

In [90]:
#generate index images
path = '/Users/aizazansari/Desktop/Uni Data/MOMA/BinaryAnalysis/images_plc/col/'
arr = os.listdir(path)
for name in arr:
    if name.endswith('.png'):
        rgb_to_index(name)

In [106]:
path = '/Users/aizazansari/Desktop/Uni Data/MOMA/BinaryAnalysis/images_plc/col_up_hilbert/'
arr = os.listdir(path)
for name in arr:
    if name.endswith('png'):
        img = Image.open(path+name)
        if img.size[0]<=512:
            crop_img(name,64)

In [105]:
def crop_img(name,new_dimension):
    load_path= 'images_plc/col_up_hilbert/'
    save_path = 'images_plc/col_up_hilbert_crop/'
    img = Image.open(load_path+name)
    old_dimension = img.size[0]
    
    if old_dimension==new_dimension:
        img.save(save_path+name,quality=100)
    
    if old_dimension > new_dimension:
        num_row = int(old_dimension/new_dimension)
        crop = pow(num_row,2)

        for i in range(crop):

            x = (i*new_dimension)%old_dimension
            y = (i//num_row)*new_dimension 

            cropped = img.crop((x,y,x+new_dimension,y+new_dimension))
            cropped.save(save_path + name.strip('.png') +  "-" + str(i) + ".png",quality=100)


In [103]:
val = open('val.txt','w')
files = os.listdir('/Users/aizazansari/Desktop/Uni Data/MOMA/BinaryAnalysis/images_plc/col_up_hilbert_index_crop')
for count, file in enumerate(files):
    if file.endswith('png'):
        val.write(file.strip('.png') +"\n")
val.close()


In [111]:
val = open('val.txt','w')
path = '/Users/aizazansari/Desktop/Uni Data/MOMA/BinaryAnalysis/images_plc/bw/'
arr = os.listdir(path)
for name in arr:
    if name.endswith('png'):
        img = Image.open(path+name)
        if img.size[0]<=256:
            val.write(name.strip('.png') +"\n")
            img.save('selectbw/'+name,quality=100)
val.close()